In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import importlib as imp

from collections import namedtuple
from random import sample, shuffle
from functools import reduce
from itertools import accumulate
from math import floor, ceil, sqrt, log, pi
from matplotlib import pyplot as plt
from tensorflow.keras import layers, utils, losses, models as mds, optimizers

if imp.util.find_spec('aggdraw'): import aggdraw
if imp.util.find_spec('tensorflow_addons'): from tensorflow_addons import layers as tfa_layers
if imp.util.find_spec('tensorflow_models'): from official.vision.beta.ops import augment as visaugment
if imp.util.find_spec('tensorflow_probability'): from tensorflow_probability import distributions as tfd

2022-03-28 14:56:57.382119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 14:56:57.382447: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Dice Loss

In [ ]:
def dice_loss(y_true, y_pred, smooth=100):        
    intersection = tf.reduce_sum(y_true * y_pred)
    coefficient = (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    return 1 - coefficient

## IoU Loss

In [ ]:
def iou_loss(from_logits=False):
    transform = lambda x: tf.math.sigmoid(x) if from_logits else x

    def iou(y_true, y_pred):
        y_pred = transform(y_pred)
        intersection = y_true * y_pred
        union = y_true + (1 - y_true)*y_pred
        return 1 - tf.reduce_sum(intersection)/tf.reduce_sum(union)
    
    return iou

## Height and Weight Loss for Bounding Boxes (Sparse Variant)

* It works with ragged true bounded boxes.
* It computes both box and non-box losses.
* Box loss is scaled with a factor of 2.
* Both the losses are normalized with the number of respective boxes.
* Box loss lies in the range [0, 2].
* Non-box loss lies in the range [0, 1].
* It handles the bounding boxes that resolve to the same grid location by double counting.

In [ ]:
def yx_to_indices(yx, size, dtype=tf.int64):
    return tf.cast(yx*(size - 1), dtype)

def compute_item_hw_loss(bboxes, predictions):
    """
        It computes the box and non-box losses for an item.

        bboxes: Bounding boxes with shape (N_BOXES, 4)
        The last dimension contains box coordinates arranged as [y_min, x_min, y_max, x_max]

        predictions: Model predictions with shape (IMG_SIZE, IMG_SIZE, 2)
        The last dimension contains the height and width values for each grid element.

        Returns:
            box_loss: A tensor of shape (N_BOXES, 1) with loss values for each box.
            box_count: A tensor with value N_BOXES.
            non_box_loss: A tensor of shape (N_BOXES, 1) with loss values for each non-box.
            non_box_count: A tensor with the number of non-boxes. Typically, it would be
            equal to IMG_SIZE*IMG_SIZE - N_BOXES. The cases where multiple bounding boxes
            resolve to the same grid pixel, this value would be:
                IMG_SIZE*IMG_SIZE - N_NON_DUPLICATE_BOXES.
    """
    bboxes = bboxes.to_tensor()
    num_boxes = tf.shape(bboxes)[0]
    size = predictions.shape[0]

    # Extract box properties.
    [yx_min, yx_max] = tf.split(bboxes, 2, axis=-1)

    # Compute box dimensions which includes their heights and widths
    hw = yx_max - yx_min

    # Compute (y_min, x_min) indices of the boxes to address their top-left corner.
    yx_indices = yx_to_indices(yx_min, size)

    # tf.print('hw: ', hw, hw.shape)
    # tf.print('yx_indices: ', yx_indices, yx_indices.shape)

    # Compute indices for sparse tensor of shape (IMG_SIZE, IMG_SIZE, 2) as [[Y, X, H], [Y, X, W]...].
    sparse_yx_indices = tf.repeat(yx_indices, 2, axis=0)
    sparse_hw_indices = tf.reshape(tf.repeat(tf.constant([[0, 1]], dtype=tf.int64), num_boxes, axis=0), [-1, 1])
    sparse_indices = tf.concat([sparse_yx_indices, sparse_hw_indices], axis=-1)
    sparse_values = tf.reshape(hw, [-1])

    # tf.print('sparse_yx_indices: ', sparse_yx_indices, sparse_yx_indices.shape)
    # tf.print('sparse_hw_indices: ', sparse_hw_indices, sparse_hw_indices.shape)
    # tf.print('sparse_indices: ', sparse_indices, sparse_indices.shape)
    # tf.print('sparse_values: ', sparse_values, sparse_values.shape)

    hw_grid = tf.sparse.reorder(tf.sparse.SparseTensor(indices=sparse_indices, values=sparse_values, dense_shape=(size, size, 2)))
    box_mask = tf.sparse.map_values(tf.ones_like, hw_grid)
    
    # The boxes which start at the same (y, x) are double counted in the box_loss.
    # The box_count refers to the total number of boxes. We normalize with box_count
    # which automatically weighs box_loss 2X because it combines both H and W.
    box_predictions = box_mask*predictions
    box_loss = tf.math.abs(tf.sparse.map_values(tf.math.subtract, hw_grid, box_predictions))
    box_loss = tf.sparse.reduce_sum(box_loss)
    box_count = tf.cast(num_boxes, tf.float32)
    # mean_box_loss = tf.sparse.reduce_sum(box_loss)/(2.0*tf.cast(num_boxes, tf.float32))

    # tf.print('hw_grid: ', hw_grid, hw_grid.shape)
    # tf.print('box_mask: ', box_mask, box_mask.shape)
    # tf.print('box_predictions: ', box_predictions, box_predictions.shape)
    # tf.print('box_loss: ', box_loss, box_loss.shape)

    # We ignore the duplicates while calculating non_box_mask values.
    non_box_mask = 1. - tf.sparse.to_dense(box_mask, validate_indices=False)
    non_box_loss = tf.math.reduce_sum(non_box_mask*predictions)
    non_box_count = tf.math.reduce_sum(non_box_mask)

    tf.print('non_box_mask: ', non_box_mask, non_box_mask.shape)
    tf.print('non_box_loss: ', non_box_loss, non_box_loss.shape)
    
    return box_loss, box_count, non_box_loss, non_box_count

def compute_hw_loss(y_true, y_pred):
    """
        It computes the mean height and width losses for boxes and non-boxes.
    """
    box_loss, box_count, non_box_loss, non_box_count = 0., 0., 0., 0.
    batch_size = tf.shape(y_true)[0]
    y_pred_hw = y_pred[:, :, :, :2]

    for item_id in range(batch_size):
        result  = compute_item_hw_loss(y_true[item_id], y_pred_hw[item_id])
        item_box_loss, item_box_count, item_non_box_loss, item_non_box_count = result
        box_loss, box_count = box_loss + item_box_loss, box_count + item_box_count
        non_box_loss, non_box_count = non_box_loss + item_non_box_loss, non_box_count + item_non_box_count
    
    mean_box_loss = box_loss / box_count
    mean_non_box_loss = non_box_loss / non_box_count
    loss = mean_box_loss + mean_non_box_loss

    # tf.print('box_loss: ', box_loss, ' box_count: ', box_count)
    # tf.print('non_box_loss: ', non_box_loss, ' non_box_count: ', non_box_count)
    # tf.print('mean_box_loss: ', mean_box_loss)
    # tf.print('mean_non_box_loss: ', mean_non_box_loss)
    # tf.print('loss: ', loss)

    return loss, mean_box_loss, mean_non_box_loss

compute_hw_loss.__name__ = 'loss'

# y_true = tf.ragged.constant([[[.4, .3, .6, .6], [.2, .2, .8, .8]], [[.2, .2, .8, .8]]])
# y_pred = tf.random.uniform((2, 4, 4, 2))

# hw_grid = tf.sparse.SparseTensor(indices=[[0, 0, 0], [2, 2, 1]], values=[2., 4.], dense_shape=(4, 4, 2))
# hw_mask = tf.sparse.map_values(tf.ones_like, hw_grid)
# # tf.sparse.to_dense(hw_grid), tf.sparse.to_dense(hw_mask*y_pred[0])
# tf.print('y_pred[0]: ', y_pred[0], y_pred[0].shape)
# compute_hw_loss(y_true, y_pred, 4)

## Height and Weight Loss for Bounding Boxes

* It computes both box and non-box losses.
* Both the losses are normalized with the number of respective boxes.
* Box and Non-box losses lie in the range [0, 2].
* It handles the bounding boxes that resolve to the same grid location by choosing the newest value.

In [ ]:
def yx_to_indices(yx, size, dtype=tf.int64):
    return tf.cast(yx*(size - 1), dtype)

def yxyx_to_hw_grid(boxes, grid_size):
    """
        It fits a tensor of flat boxes to a grid of given size.

        Arguments:
            boxes: A tensor of boxes in YXYX format with shape (N_BOXES, 4)
            grid_size: An integer value indicating the size of the target grid.
        
        Returns:
            A grid containing heights and widths of input boxes fitted based on their top-left coordinates.
            The output shape is (grid_size, grid_size, 2)
    """
    yx_min, yx_max = tf.split(boxes, 2, axis=-1)
    hw = yx_max - yx_min

    box_indices = yx_to_indices(yx_min, grid_size, dtype=tf.int32)
    box_grid = tf.scatter_nd(indices=box_indices, updates=hw, shape=(grid_size, grid_size, 2))
    
    return box_grid

@tf.function
def make_random_grid_boxes(num_boxes, grid_size):
    """
        Generates a random boxes in a grid.

        Arguments:
            num_boxes: An integer value to indicate the number of boxes to generate.
            grid_size: The size of target grid to fit the boxes into.
        
        Returns:
            A grid containing heights and widths of input boxes fitted based on their top-left coordinates.
            The output shape is (grid_size, grid_size, 2)
    """
    pair_1 = tf.random.uniform((num_boxes, 2))
    pair_2 = tf.random.uniform((num_boxes, 2))

    yx_min = tf.where(pair_1 < pair_2, pair_1, pair_2)
    yx_max = tf.where(pair_1 > pair_2, pair_1, pair_2)

    boxes = tf.concat([yx_min, yx_max], axis=-1)
    box_grid = yxyx_to_hw_grid(boxes, grid_size)

    # tf.print('box_indices: ', box_indices, box_indices.shape)
    # tf.print('boxes: ', boxes, boxes.shape)
    # tf.print('box_grid: ', box_grid, box_grid.shape)

    return box_grid

@tf.function
def compute_item_hw_loss(bboxes, predictions):
    """
        bboxes: A tensor of boxes with shape (IMG_SIZE, IMG_SIZE, 2)
        The last dimension contains heights and widths of the boxes.

        predictions: A tensor of boxes with shape (IMG_SIZE, IMG_SIZE, 2)
        The last dimension contains heights and widths of the boxes.

        Returns:
            * Box Loss
            * Box Count
            * Non-box Loss
            * Non-box Count
    """
    # The box_count refers to the total number of boxes. We normalize with box_count
    # which automatically weighs box_loss 2X because it combines both H and W.
    box_mask = tf.cast(tf.math.reduce_all(tf.math.not_equal(bboxes, 0.), axis=-1, keepdims=True), tf.float32)
    box_predictions = box_mask*predictions
    box_loss = tf.math.reduce_sum(tf.math.abs(bboxes - box_predictions))
    box_count = tf.math.reduce_sum(box_mask)

    # display('box_mask', box_mask)
    # display('box_predictions', box_predictions)
    # display('box_loss', box_loss)
    # display('box_count', box_count)

    non_box_mask = 1 - box_mask
    non_box_predictions = non_box_mask*predictions
    non_box_loss = tf.math.reduce_sum(non_box_predictions)
    non_box_count = tf.math.reduce_sum(non_box_mask)

    # display('non_box_mask', non_box_mask)
    # display('non_box_predictions', non_box_predictions)
    # display('non_box_loss', non_box_loss)
    # display('non_box_count', non_box_count)

    return [box_loss, box_count, non_box_loss, non_box_count]

def compute_hw_loss(y_true, y_pred):
    box_loss, box_count, non_box_loss, non_box_count = 0., 0., 0., 0.
    batch_size = tf.shape(y_true)[0]
    y_pred_hw = y_pred[:, :, :, :2]

    for item_id in range(batch_size):
        result  = compute_item_hw_loss(y_true[item_id], y_pred_hw[item_id])
        item_box_loss, item_box_count, item_non_box_loss, item_non_box_count = result
        box_loss, box_count = box_loss + item_box_loss, box_count + item_box_count
        non_box_loss, non_box_count = non_box_loss + item_non_box_loss, non_box_count + item_non_box_count
    
    mean_box_loss = box_loss / box_count
    mean_non_box_loss = non_box_loss / non_box_count
    loss = mean_box_loss + mean_non_box_loss

    # tf.print('box_loss: ', box_loss, ' box_count: ', box_count)
    # tf.print('non_box_loss: ', non_box_loss, ' non_box_count: ', non_box_count)
    # tf.print('mean_box_loss: ', mean_box_loss)
    # tf.print('mean_non_box_loss: ', mean_non_box_loss)
    # tf.print('loss: ', loss)

    return loss, mean_box_loss, mean_non_box_loss

compute_hw_loss.__name__ = 'loss'

# y_true = tf.map_fn(lambda v: make_random_grid_boxes(v[0], v[1]), [tf.constant([2, 3]), tf.ones((2))*4], fn_output_signature=tf.float32)
# y_pred = tf.random.uniform((2, 4, 4, 2))
# # y_true = tf.map_fn(lambda v: make_random_grid_boxes(v[0], v[1]), [tf.constant([4]), tf.ones((1))*4], fn_output_signature=tf.float32)
# # y_pred = tf.random.uniform((1, 4, 4, 2))
# display('y_true', y_true[0])
# display('y_pred', y_pred[0])
# compute_hw_loss(y_true, y_pred)